# lite

In [ ]:
import json
import os
import re
from pathlib import Path

import importnb

with importnb.Notebook():
    from jupyak.tasks import _actions as A
    from jupyak.tasks import _py as P
    from jupyak.tasks import _well_known as W
    from jupyak.tasks import _yak as Y

In [ ]:
def lite_tasks(yak: Y.Yak):
    work_path = yak.work_path
    lite = yak.lite.work_path
    build_conf = yak.lite.build_config_path
    in_work_lite = {"cwd": lite}
    run_args = yak.env.run_args
    lite_args = [*run_args, "jupyter", "lite"]

    all_deps = cli_deps, build_deps, run_deps = _find_lite_deps(yak)

    gist_dep = []

    if yak.lite.gist:
        gist_dep += [yak.lite.gist_path / ".git/refs/heads" / W.WORK_BRANCH]

    self_dist = os.environ.get(W.ENV_VAR_SELF_DIST)

    if self_dist:
        run_deps += [Path(self_dist)]

    yield dict(
        name="config",
        doc="> configure jupyterlite",
        file_dep=[*build_deps, *run_deps, *gist_dep],
        actions=[(_make_lite_config, [build_deps, run_deps, yak])],
        targets=[build_conf],
    )

    yield dict(
        name="build",
        doc="> build lite site archive",
        file_dep=[
            *sum(all_deps, []),
            *gist_dep,
            build_conf,
            yak.env.lab_share / "static" / W.PACKAGE_JSON,
        ],
        actions=[
            A.run([*lite_args, "build"], in_work_lite),
            A.run([*lite_args, "archive"], in_work_lite),
        ],
        targets=[yak.lite.app_shasums_path],
    )

In [ ]:
def _find_lite_deps(yak: Y.Yak):
    cli_deps = []
    build_deps = []
    run_deps = []
    for name, repo in yak.py_repos.items():
        if not repo.lite:
            continue

        cli_deps += [
            yak.env.py_site_packages / P._pth_path(pth) for pth in repo.lite.needs_pth
        ]
        for ext_js_path, ext in repo.py.lab_extensions.items():
            build_deps += [repo.work_path / t for t in ext.targets]
        if repo.lite.wheel:
            for ppt in repo.py.pyproject_tomls:
                ppt_path = repo.work_path / ppt
                if any(
                    re.match(swp, ppt_path.parent.name)
                    for swp in repo.lite.skip_wheel_patterns
                ):
                    continue
                run_deps += [ppt_path.parent / "dist" / W.SHA256SUMS]
    return cli_deps, build_deps, run_deps

In [ ]:
def _make_lite_config(build_deps, run_deps, yak: Y.Yak, config=None):
    work_path = yak.lite.work_path
    work_path.mkdir(exist_ok=True, parents=True)
    contents = []
    config = config or {}
    lbc = config.setdefault("LiteBuildConfig", {})
    pla = config.setdefault("PipliteAddon", {})
    pkg_jsons = [
        p
        for p in build_deps
        if p.name == W.PACKAGE_JSON
        and (p.parent / "static" / W.THIRD_PARTY_LICENSES).exists()
    ]
    whl_deps = [
        p
        for p in run_deps
        if p.name == W.SHA256SUMS and p.parent.glob(f"*{W.NOARCH_WHL}")
    ]

    if yak.lite.gist:
        contents = [os.path.relpath(yak.lite.gist_path, work_path)]
        gist_config_path = yak.lite.gist_path / W.JUPYTER_LITE_CONFIG
        if gist_config_path.exists():
            gist_config = json.loads(gist_config_path.read_text(encoding="utf-8"))
            for has_traits, trait_values in gist_config.items():
                config.setdefault(has_traits, {}).update(trait_values)
        for run_path in [W.JUPYTER_LITE_JSON, W.JUPYTER_LITE_IPYNB]:
            src = yak.lite.gist_path / run_path
            if not src.exists():
                continue
            dest = work_path / run_path
            dest.write_text(src.read_text(encoding="utf-8"), encoding="utf-8")

    lbc.update(
        cache_dir="../.cache/lite",
        contents=contents,
        output_dir=os.path.relpath(yak.lite.app_path, work_path),
        ignore_sys_prefix=["federated_extensions"],
        federated_extensions=[
            os.path.relpath(pkg_json.parent, work_path) for pkg_json in pkg_jsons
        ],
    )

    pla.update(
        piplite_urls=[
            *pla.get("piplite_urls", []),
            *[os.path.relpath(whl_shas.parent, work_path) for whl_shas in whl_deps],
        ]
    )

    yak.lite.build_config_path.write_text(json.dumps(config, indent=2, sort_keys=True))